In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1, 1, 28, 28)
print(f'Tensor 크기 : {inputs.shape}')

Tensor 크기 : torch.Size([1, 1, 28, 28])


In [3]:
conv1 = nn.Conv2d(1, 32, 3, padding = 1)
print(conv1)

conv2 = nn.Conv2d(32, 64, kernel_size=3, padding = 1)
print(conv2)

pool = nn.MaxPool2d(2)
print(pool)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [7]:
out = conv1(inputs)
print(out.shape)

out = pool(out)
print(out.shape)

out = conv2(out)
print(out.shape)

out = pool(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 32, 14, 14])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [9]:
print(out.size(0))
print(out.size(1))
print(out.size(2))
print(out.size(3))

1
64
7
7


In [10]:
# 첫번째 차원(배치차원)은 그대로 두고 나머지 펼치기
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [11]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape) # 출력층으로 10 개 뉴런 --> 차원 텐서 변환

torch.Size([1, 10])


# CNN으로 Mnist 분류하기 
---

In [12]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(0)
if device == 'cuda' :
    torch.cuda.manual_seed_all(0)

In [14]:
lr = 0.001
epochs = 15
batch_size = 100

In [15]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [21]:
class CNN(torch.nn.Module) :
    
    def __init__(self) :
        super(CNN, self).__init__()
        
        # 1st layer
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 2nd layer
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # 전결합층 (7*7*64 --> 10)
        self.fc = torch.nn.Linear(7*7*64, 10, bias=True)
        
        # 전결합층 한정으로 가중치를 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    
    def forward(self, x) :
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [22]:
model = CNN().to(device)

In [23]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [24]:
total_batch = len(data_loader)
print(f'총 배치 수 : {total_batch}')

총 배치 수 : 600


In [25]:
for epoch in range(epochs) :
    avg_cost = 0
    
    for X, y in data_loader :
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        h = model(X)
        cost = criterion(h, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print(f'[Epoch : {epoch+1:>4}] cost = {avg_cost:>.9}')

Epoch :    1] cost = 0.228284776
Epoch :    2] cost = 0.0627280697
Epoch :    3] cost = 0.0458956547
Epoch :    4] cost = 0.0369985849
Epoch :    5] cost = 0.0293225739
Epoch :    6] cost = 0.025048079
Epoch :    7] cost = 0.0212471951
Epoch :    8] cost = 0.0180564579
Epoch :    9] cost = 0.0152841294
Epoch :   10] cost = 0.0139708081
Epoch :   11] cost = 0.0110666752
Epoch :   12] cost = 0.0087069897
Epoch :   13] cost = 0.00916985795
Epoch :   14] cost = 0.00741479965
Epoch :   15] cost = 0.00696938531


In [27]:
with torch.no_grad() :
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = model(X_test)
    corr_pred = torch.argmax(pred, 1) == y_test
    acc = corr_pred.float().mean()
    print(f'Accuracy : {acc.item()}')

Accuracy : 0.9865999817848206


# DEEP CNN으로 Mnist 분류하기 
---

In [ ]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(0)

if device == 'cuda' :
    torch.cuda.manual_seed_all(0)


In [ ]:
lr = 0.001
epochs = 15
batch_size= 100

In [ ]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)


data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [33]:
class CNN(torch.nn.Module) :
    
    def __init__(self) :
        super(CNN, self).__init__()
        self.keep_prob = 0.5
    
        self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size= 2, stride= 2))
        
        self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size= 3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4*4*128,625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1, 
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1-self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    
    def forward(self, x) :
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

model = CNN().to(device)

In [34]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

total_batch = len(data_loader)
print(f'총 배치의 수 : {total_batch}')

총 배치의 수 : 600


In [35]:
for epoch in range(epochs) :
    avg_cost = 0
    
    for X, y in data_loader :
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        h = model(X)
        cost = criterion(h, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print(f'[Epoch : {epoch+1:>4}] cost = {avg_cost:>.9}')
        

[Epoch :    1] cost = 0.184816927
[Epoch :    2] cost = 0.0499588661
[Epoch :    3] cost = 0.0366500728
[Epoch :    4] cost = 0.0287286565
[Epoch :    5] cost = 0.023685623
[Epoch :    6] cost = 0.0201001558
[Epoch :    7] cost = 0.014927269
[Epoch :    8] cost = 0.0132745868
[Epoch :    9] cost = 0.0112704094
[Epoch :   10] cost = 0.0118605439
[Epoch :   11] cost = 0.0107971095
[Epoch :   12] cost = 0.00915571302
[Epoch :   13] cost = 0.00905944686
[Epoch :   14] cost = 0.00666494528
[Epoch :   15] cost = 0.00827543531


In [36]:
with torch.no_grad() :
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = model(X_test)
    corr_pred = torch.argmax(pred, 1) == y_test
    acc = corr_pred.float().mean()
    print(f'Accuracy : {acc.item()}')

C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\torchvision\datasets\mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy : 0.9624999761581421
